In [1]:
import pandas as pd
import numpy as np
import quandl
# ! pip install wrds

In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

Get Tickers for most active stock options

In [3]:
PATH_OPTIONS_DATA_1 = r"industry1_6_option.csv"
PATH_OPTIONS_DATA_2 = r"industry6_11.csv"

In [4]:
options_data_1_6 = pd.read_csv(PATH_OPTIONS_DATA_1)
options_data_1_6.head()

,secid,date,days,impl_volatility,impl_strike,cp_flag,ticker,index_flag
0,100972,2010-01-04,10,NaN,0.0,P,ABT,0
1,100972,2010-01-04,10,NaN,0.0,P,ABT,0
2,100972,2010-01-04,10,NaN,0.0,P,ABT,0
3,100972,2010-01-04,10,NaN,0.0,P,ABT,0
4,100972,2010-01-04,10,NaN,0.0,P,ABT,0


In [5]:
options_data_1_6['date'] = pd.to_datetime(options_data_1_6['date'])

In [6]:
options_data_1_6['expiration'] = options_data_1_6['date'] + pd.to_timedelta(options_data_1_6['days'], unit='D')

In [7]:
# options_6_11 = pd.read_csv(PATH_OPTIONS_DATA_2)
# options_6_11.head()

In [8]:
# options_6_11['date'] = pd.to_datetime(options_6_11['date'])

In [9]:
# options_6_11['expiration'] = options_6_11['date'] + pd.to_timedelta(options_6_11['days'], unit='D')

In [10]:
import os

def grab_quandl_table(
    table_path,
    avoid_download=False,
    replace_existing=False,
    date_override=None,
    allow_old_file=False,
    **kwargs,
):
    root_data_dir = os.path.join(os.environ["HOME"], "Downloads")
    data_symlink = os.path.join(root_data_dir, f"{table_path}_latest.zip")
    if avoid_download and os.path.exists(data_symlink):
        print(f"Skipping any possible download of {table_path}")
        return data_symlink
    
    table_dir = os.path.dirname(data_symlink)
    if not os.path.isdir(table_dir):
        print(f'Creating new data dir {table_dir}')
        os.mkdir(table_dir)

    if date_override is None:
        my_date = datetime.datetime.now().strftime("%Y%m%d")
    else:
        my_date = date_override
    data_file = os.path.join(root_data_dir, f"{table_path}_{my_date}.zip")

    if os.path.exists(data_file):
        file_size = os.stat(data_file).st_size
        if replace_existing or not file_size > 0:
            print(f"Removing old file {data_file} size {file_size}")
        else:
            print(
                f"Data file {data_file} size {file_size} exists already, no need to download"
            )
            return data_file

    dl = quandl.export_table(
        table_path, filename=data_file, api_key="TDynpHNbX5ucpahcz_85"
, **kwargs
    )
    file_size = os.stat(data_file).st_size
    if os.path.exists(data_file) and file_size > 0:
        print(f"Download finished: {file_size} bytes")
        if not date_override:
            if os.path.exists(data_symlink):
                print(f"Removing old symlink")
                os.unlink(data_symlink)
            print(f"Creating symlink: {data_file} -> {data_symlink}")
            os.symlink(
                data_file, data_symlink,
            )
    else:
        print(f"Data file {data_file} failed download")
        return
    return data_symlink if (date_override is None or allow_old_file) else "NoFileAvailable"


def fetch_quandl_table(table_path, avoid_download=True, **kwargs):
    return pd.read_csv(
        grab_quandl_table(table_path, avoid_download=avoid_download, **kwargs)
    )

In [11]:
prices_eod_primary = fetch_quandl_table('QUOTEMEDIA/PRICES', avoid_download=True)
prices_eod_primary['date'] = pd.to_datetime(prices_eod_primary['date'])
prices_eod_primary.head()

Skipping any possible download of QUOTEMEDIA/PRICES


,ticker,date,open,high,low,close,volume,dividend,split,adj_open,adj_high,adj_low,adj_close,adj_volume
0,JTKWY,2022-03-11,6.1700,7.3200,5.7900,6.72,9440097.0,0.0,1.0,6.1700,7.3200,5.7900,6.72,9440097.0
1,JTKWY,2022-03-10,6.1600,6.1750,5.9350,6.07,2261623.0,0.0,1.0,6.1600,6.1750,5.9350,6.07,2261623.0
2,FG_1,2020-06-01,8.1000,8.3900,8.1000,8.39,3086317.0,0.0,1.0,8.1000,8.3900,8.1000,8.39,3086317.0
3,YTENQ,2024-09-30,0.9514,1.0500,0.9514,1.05,842.0,0.0,1.0,0.9514,1.0500,0.9514,1.05,842.0
4,FLWS,2022-03-09,14.5700,14.9588,14.4100,14.45,662492.0,0.0,1.0,14.5700,14.9588,14.4100,14.45,662492.0


In [12]:
industries = pd.read_csv('/Users/mahimaraut/UChicago/qts/project/buy-rough-sell-smooth/industry.csv')

In [13]:
industries.head()

,industry,top_tickers
0,CRSP US Consumer Discretionary Index,AMZN TSLA HD WMT MCD BKNG LOW TJX UBER...
1,CRSP US Consumer Staples Index,COST PG KO PEP PM MO MDLZ CL KMB KVUE
2,CRSP US Energy Index,XOM CVX COP EOG WMB OKE SLB KMI PSX LNG
3,CRSP US Financials Index,BRK.A/BRK.B JPM V MA BAC WFC GS SPGI M...
4,CRSP US Healthcare Index,LLY UNH JNJ ABBV MRK TMO ABT ISRG PFE ...


In [14]:
industries['industry'] = industries.industry.str.replace('CRSP ', '', regex=True)
industries['industry'] = industries.industry.str.replace(' Index', '', regex=True)
industries['top_tickers'] = industries['top_tickers'].str.split()
industries = industries.explode('top_tickers').reset_index(drop=True)
industries

,industry,top_tickers
0,US Consumer Discretionary,AMZN
1,US Consumer Discretionary,TSLA
2,US Consumer Discretionary,HD
3,US Consumer Discretionary,WMT
4,US Consumer Discretionary,MCD
...,...,...
105,US Utilities,AEP
106,US Utilities,VST
107,US Utilities,D
108,US Utilities,PCG


In [15]:
industries.rename(columns={'top_tickers': 'tickers'}, inplace=True)
industries

,industry,tickers
0,US Consumer Discretionary,AMZN
1,US Consumer Discretionary,TSLA
2,US Consumer Discretionary,HD
3,US Consumer Discretionary,WMT
4,US Consumer Discretionary,MCD
...,...,...
105,US Utilities,AEP
106,US Utilities,VST
107,US Utilities,D
108,US Utilities,PCG


In [16]:
ticker_list = industries['tickers'].tolist()


In [17]:
# options_dict = {
#     ticker: {
#         date: {
#             expiration: sub_df.reset_index(drop=True)
#             for expiration, sub_df in date_df.groupby('expiration')
#         }
#         for date, date_df in ticker_df.groupby('date')
#     }
#     for ticker, ticker_df in options_data_1_6.groupby('ticker')
# }

options_dict = { ticker : options_data_1_6[options_data_1_6['ticker'] == ticker] for ticker in ticker_list }


In [18]:
options_dict['UBER']

,secid,date,days,impl_volatility,impl_strike,cp_flag,ticker,index_flag,expiration
43569878,213638,2019-05-16,10,0.557330,48.57762,P,UBER,0,2019-05-26
43569879,213638,2019-05-16,10,0.560278,47.53992,P,UBER,0,2019-05-26
43569880,213638,2019-05-16,10,0.563888,46.72538,P,UBER,0,2019-05-26
43569881,213638,2019-05-16,10,0.567750,46.03078,P,UBER,0,2019-05-26
43569882,213638,2019-05-16,10,0.571196,45.40570,P,UBER,0,2019-05-26
...,...,...,...,...,...,...,...,...,...
60199035,213638,2023-08-31,730,0.431894,45.94319,C,UBER,0,2025-08-30
60199036,213638,2023-08-31,730,0.439786,41.88845,C,UBER,0,2025-08-30
60199037,213638,2023-08-31,730,0.450169,37.63254,C,UBER,0,2025-08-30
60199038,213638,2023-08-31,730,0.463555,32.99751,C,UBER,0,2025-08-30


In [19]:
import json 

filename = "options_6_11.json"

with open(filename, "w") as file:
    json.dumps(options_dict, file, indent=4)

TypeError: dumps() takes 1 positional argument but 2 positional arguments (and 1 keyword-only argument) were given

In [ ]:
# options_dict1 = {
#     {
#         ticker: {
#             date: {
#                 expiration: sub_df.reset_index(drop=True)
#                 for expiration, sub_df in date_df.groupby('expiration')
#             }
#             for date, date_df in ticker_df.groupby('date')
#         }
#         for ticker, ticker_df in options_6_11.groupby('ticker')
#     }
# }


In [ ]:
row = options_dict.keys()
row


dict_keys(['A', 'ABBV', 'ABT', 'AMZN', 'AXP', 'B', 'BAC', 'BKNG', 'BRK', 'CL', 'COP', 'COST', 'CVX', 'EOG', 'GS', 'HD', 'JNJ', 'JPM', 'KMB', 'KMI', 'KO', 'KVUE', 'LLY', 'LNG', 'LOW', 'MA', 'MCD', 'MDLZ', 'MO', 'MRK', 'MS', 'OKE', 'PEP', 'PG', 'PM', 'PSX', 'SBUX', 'SLB', 'SPGI', 'TJX', 'TMO', 'TSLA', 'UBER', 'UNH', 'V', 'WFC', 'WMB', 'WMT', 'XOM'])

In [ ]:
# close_value = prices_eod_primary[(prices_eod_primary['ticker'] == 'GOOGL') & (prices_eod_primary['date'] == '2019-01-02')]
# close_value

In [ ]:
# # sample pull
# for ticker in options_dict.keys():
#     options_dict[ticker]['secid'] = options_dict[ticker]['secid'].astype(str).str.split(' ').str[0]
# row = options_dict['AMZN'][options_dict['AMZN']['date'] == '2019-01-02']
# row


In [ ]:
current_year = 2023

In [ ]:
def power_law(tau, C, H):
    """ Power-law function: phi(tau) = C * tau^(H - 1/2) """
    return C * tau**(H - 0.5)

In [ ]:
def get_atm_skew_slope(ticker, date, stock_price, options_data, method='mid', flag_plot=False):
    # Create a copy of options_data to avoid modifying the original DataFrame
    options_data = options_data.copy()
    
    # Use .loc[] for assignment to avoid ambiguity
    options_data.loc[:, 'impl_strike'] = pd.to_numeric(options_data['impl_strike'], errors='coerce')
    options_data.loc[:, 'impl_volatility'] = pd.to_numeric(options_data['impl_volatility'], errors='coerce')

    # Rest of your code remains the same...
    
    # Separate calls and puts
    call_options = options_data[options_data['cp_flag'] == 'C'].dropna(subset=['impl_volatility'])
    put_options = options_data[options_data['cp_flag'] == 'P'].dropna(subset=['impl_volatility'])

    # Compute log-moneyness for calls and puts
    call_options['log(K/S)'] = np.log(call_options['impl_strike'].astype(float) / stock_price['close'].iloc[0])
    put_options['log(K/S)'] = np.log(put_options['impl_strike'].astype(float) / stock_price['close'].iloc[0])

    # Sort data for proper spline fitting
    call_options = call_options.sort_values(by='log(K/S)')
    put_options = put_options.sort_values(by='log(K/S)')

    # Extract values
    log_ks_calls = call_options['log(K/S)'].values
    vols_calls = call_options['impl_volatility'].values
    
    log_ks_puts = put_options['log(K/S)'].values
    vols_puts = put_options['impl_volatility'].values

    # Fit cubic splines separately for calls and puts
    spline_calls = CubicSpline(log_ks_calls, vols_calls)
    spline_puts = CubicSpline(log_ks_puts, vols_puts)

    # Define a common grid around ATM
    grid = np.linspace(-0.2, 0.2, 100)

    # Evaluate both splines on the grid and take their mid implied volatility values
    mid_iv = (spline_calls(grid) + spline_puts(grid)) / 2

    # Fit a cubic spline to the mid implied volatility curve
    spline_mid = CubicSpline(grid, mid_iv)

    # Compute the ATM skew slope (first derivative at log(K/S) = 0)
    atm_skew_slope = spline_mid.derivative()(0).item()

    return atm_skew_slope

In [ ]:

def get_time_to_expiry(expiration, date):
    """Compute time to expiry in years, ensuring both dates are Timestamp objects."""
    expiration = pd.Timestamp(expiration)  # Ensure it's a Timestamp
    date = pd.Timestamp(date)  # Ensure it's a Timestamp

    return (expiration - date).days / 365.0  # Convert days to years

In [ ]:
from joblib import Parallel, delayed

def get_h_value(ticker, date, stock_price, options_data, flag_plot_image=False):
    # Filter out expirations that are the same as the date
    unique_expirations = options_data[options_data['expiration'] != date]['expiration'].unique()
    # unique_expirations = options_data.keys()

    # Initialize lists to store results
    expiration_phi_pairs = []

    # # Loop through each expiration
    # def process_expiration(expiration):
    #     print(f'Ticker: {ticker}, Date: {date}, Expiration: {expiration}')
        
    #     # Filter options data for the current expiration
    #     # options_data_filtered = options_data[options_data['expiration'] == expiration]
    #     options_data_filtered = options_data[expiration]
        
    #     # Compute phi using get_atm_skew_slope function
    #     phi = get_atm_skew_slope(ticker, date, stock_price, options_data_filtered, "mid", False)
        
    #     # Calculate time to expiry
    #     time_to_expiry = options_data_filtered['days'].iloc[0] / 365
        
    #     return (time_to_expiry, phi)

    # # Parallelize the processing of expirations
    # expiration_phi_pairs = Parallel(n_jobs=-1, verbose=10)(
    #     delayed(process_expiration)(expiration) for expiration in unique_expirations
    # )

    for expiration in unique_expirations:
        print(f'Ticker: {ticker}, Date: {date}, Expiration: {expiration}')
        
        # Filter options data for the current expiration
        options_data_filtered = options_data[options_data['expiration'] == expiration]
        # options_data_filtered = options_data[expiration]    

        phi = get_atm_skew_slope(ticker, date, stock_price, options_data_filtered, "mid", False)
        
        # Calculate time to expiry
        time_to_expiry = options_data_filtered['days'].iloc[0] / 365

        expiration_phi_pairs.append((time_to_expiry, phi))

    # Convert list to NumPy arrays for vectorized operations
    expirations, phis = zip(*expiration_phi_pairs)
    expirations = np.array(expirations)
    phis = np.array(phis)
    abs_phis = np.abs(phis)

    # Fit the power-law model to the absolute values of phis
    popt, _ = curve_fit(power_law, expirations, abs_phis, p0=(1, 0.5), maxfev=10000)
    C_opt, H_opt = popt
    fitted_phi = power_law(expirations, C_opt, H_opt)

    if flag_plot_image:
        # Plot original data
        plt.figure(figsize=(10, 6))
        plt.plot(expirations, abs_phis, marker='o', linestyle='-', color='b', label="|Phi| vs Expiration")
        
        # Plot fitted power-law curve
        plt.plot(expirations, fitted_phi, linestyle="--", color="r", label="Fitted Power-Law Curve")
        
        plt.xlabel("Time to Expiration (years)")
        plt.ylabel("|Phi|")
        plt.title("Power-Law Fit: |Phi| vs Expiration")
        plt.legend()
        plt.grid(True)
        plt.show()

    print(f"Optimized H Value: {H_opt}")
    return H_opt

In [ ]:
# ```

# import pandas as pd
# import pandas_market_calendars as mcal

# # Get the trading calendar for NYSE
# nyse = mcal.get_calendar('NYSE')

# # ticker_list = ['AMZN']
# current_year = 2023  # Ensure `current_year` is defined
# year_list = range(2023, 2024)  # Example: from 2020 to 2023

# # Placeholder list for H-values
# h_values_list = []

# for ticker in ticker_list:
#     for year in year_list:

#         # Get trading dates for the specified year
#         trading_dates = nyse.valid_days(start_date=f'{current_year}-01-01', end_date=f'{current_year}-12-31')
#         series = pd.Series(trading_dates.to_pydatetime())  # Convert to a Series
#         series = series.dt.tz_localize(None).dt.normalize()  # Ensure it's timezone naive

#         for date in series:
#             try:
#                 print(f'Ticker: {ticker}, Date: {date}')
#                 options_data = options_dict[ticker][options_dict[ticker]['date'] == date]
#             except KeyError as e:
#                 print(f"KeyError fetching options data for {ticker} on {date}: {e}")
#                 continue
#             except Exception as e:
#                 print(f"Error fetching options data for {ticker} on {date}: {e}")
#                 continue
            
#             close_value = prices_eod_primary[(prices_eod_primary['ticker'] == ticker) & (prices_eod_primary['date'] == date)]
            
#             try:
#                 h_value = get_h_value(ticker, date, close_value, options_data, False)  # Ensure `get_h_value` is defined
#             except Exception as e:
#                 print(f"Error calculating H value for {ticker} on {date}: {e}")
#                 continue

#             # Append the new row to the list

#             h_values_list.append({'Ticker': ticker, 'Date': date, 'Price': close_value['close'].iloc[0], 'H_value': h_value})
       

# # Create the DataFrame from the list
# h_values_df = pd.DataFrame(h_values_list)

# ```

In [ ]:
import pandas as pd
import pandas_market_calendars as mcal
from joblib import Parallel, delayed
import numpy as np

# Get the trading calendar for NYSE
nyse = mcal.get_calendar('NYSE')

def process_ticker_year(ticker, year):
    # Get trading dates for the specified year
    trading_dates = nyse.valid_days(start_date=f'{year}-01-01', end_date=f'{year}-12-31')
    series = pd.Series(trading_dates.to_pydatetime())  # Convert to a Series
    series = series.dt.tz_localize(None).dt.normalize()  # Ensure it's timezone naive

    h_values_list = []

    for date in series:
        try:
            print(f'Ticker: {ticker}, Date: {date}')
            options_data = options_dict[ticker][options_dict[ticker]['date'] == date]
            # options_data = options_dict[ticker][date]
        except KeyError as e:
            print(f"KeyError fetching options data for {ticker} on {date}: {e}")
            continue
        except Exception as e:
            print(f"Error fetching options data for {ticker} on {date}: {e}")
            continue
        
        close_value = prices_eod_primary[(prices_eod_primary['ticker'] == ticker) & (prices_eod_primary['date'] == date)]
        
        try:
            h_value = get_h_value(ticker, date, close_value, options_data, False)  # Ensure `get_h_value` is defined
        except Exception as e:
            print(f"Error calculating H value for {ticker} on {date}: {e}")
            continue

        # Append the new row to the list
        h_values_list.append({'Ticker': ticker, 'Year': year, 'Date': date, 'Price': close_value['close'].iloc[0], 'H_value': h_value})

    return h_values_list

def main():
    # ticker_list = ['AMZN']
    # ticker_list = options_dict.keys()
    year_list = range(2010, 2024)  # Example: from 2020 to 2023

    # Generate all combinations of tickers and years
    args = [(ticker, year) for ticker in ticker_list for year in year_list]
    
    # Run the function in parallel for each combination
    results = Parallel(n_jobs=-1, verbose=10)(
        delayed(process_ticker_year)(ticker, year) for ticker, year in args
    )
    print(f"Number of active jobs running in parallel: {len(results)}")

    # Flatten the results into a single list
    h_values_list = [item for sublist in results for item in sublist]

    # Create the DataFrame from the list
    h_values_df = pd.DataFrame(h_values_list)

    return h_values_df

# Run the main function
h_values_df = main()


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
/Users/mahimaraut/miniconda3/envs/trading/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [ ]:
h_values_df

,Ticker,Year,Date,Price,H_value
0,AMZN,2020,2020-01-02,1898.01,-0.188426
1,AMZN,2020,2020-01-03,1874.97,0.359484
2,AMZN,2020,2020-01-06,1902.88,0.165093
3,AMZN,2020,2020-01-07,1906.86,0.108744
4,AMZN,2020,2020-01-08,1891.97,0.149535
...,...,...,...,...,...
918,AMZN,2023,2023-08-25,133.26,0.118268
919,AMZN,2023,2023-08-28,133.14,0.123448
920,AMZN,2023,2023-08-29,134.91,0.203156
921,AMZN,2023,2023-08-30,135.07,0.272739


In [ ]:
ticker_list

['AMZN']

In [ ]:
h_values_df

,Ticker,Year,Date,Price,H_value
0,AMZN,2020,2020-01-02,1898.01,-0.188426
1,AMZN,2020,2020-01-03,1874.97,0.359484
2,AMZN,2020,2020-01-06,1902.88,0.165093
3,AMZN,2020,2020-01-07,1906.86,0.108744
4,AMZN,2020,2020-01-08,1891.97,0.149535
...,...,...,...,...,...
918,AMZN,2023,2023-08-25,133.26,0.118268
919,AMZN,2023,2023-08-28,133.14,0.123448
920,AMZN,2023,2023-08-29,134.91,0.203156
921,AMZN,2023,2023-08-30,135.07,0.272739


In [ ]:
h_values_df.sort_values(by='Date', inplace=True)

In [ ]:
h_values_df

,Ticker,Year,Date,Price,H_value
0,AMZN,2020,2020-01-02,1898.01,-0.188426
1,AMZN,2020,2020-01-03,1874.97,0.359484
2,AMZN,2020,2020-01-06,1902.88,0.165093
3,AMZN,2020,2020-01-07,1906.86,0.108744
4,AMZN,2020,2020-01-08,1891.97,0.149535
...,...,...,...,...,...
918,AMZN,2023,2023-08-25,133.26,0.118268
919,AMZN,2023,2023-08-28,133.14,0.123448
920,AMZN,2023,2023-08-29,134.91,0.203156
921,AMZN,2023,2023-08-30,135.07,0.272739


In [ ]:
h_values_df.reset_index(drop=True, inplace=True)

In [ ]:
h_values_df

,Ticker,Year,Date,Price,H_value
0,AMZN,2020,2020-01-02,1898.01,-0.188426
1,AMZN,2020,2020-01-03,1874.97,0.359484
2,AMZN,2020,2020-01-06,1902.88,0.165093
3,AMZN,2020,2020-01-07,1906.86,0.108744
4,AMZN,2020,2020-01-08,1891.97,0.149535
...,...,...,...,...,...
918,AMZN,2023,2023-08-25,133.26,0.118268
919,AMZN,2023,2023-08-28,133.14,0.123448
920,AMZN,2023,2023-08-29,134.91,0.203156
921,AMZN,2023,2023-08-30,135.07,0.272739


In [ ]:
# Ensure 'Date' is in datetime format
h_values_df['Date'] = pd.to_datetime(h_values_df['Date'])

# Resample using both 'Ticker' and 'Date' grouped by month
monthly_h_values_df = (
    h_values_df
    .groupby(['Ticker', pd.Grouper(key='Date', freq='M')])  # Resample each ticker by month-end
    .agg({
        'Price': 'last',  # Last price of the month
        'H_value': 'mean'  # Mean H-value for the month
    })
    .reset_index()
)




/var/folders/7c/fms9v7rj3w3_ssh2tnrr_0t00000gn/T/ipykernel_48885/381736551.py:7: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .groupby(['Ticker', pd.Grouper(key='Date', freq='M')])  # Resample each ticker by month-end


In [ ]:
monthly_h_values_df.to_csv('sample_implied_H_values_2023.csv', index=False)

In [ ]:
monthly_h_values_df

,Ticker,Date,Price,H_value
0,AMZN,2020-01-31,2008.72,0.335932
1,AMZN,2020-02-29,1883.75,0.142785
2,AMZN,2020-03-31,1949.72,-0.040043
3,AMZN,2020-04-30,2474.00,0.229690
4,AMZN,2020-05-31,2442.37,0.242886
5,AMZN,2020-06-30,2758.82,0.318169
6,AMZN,2020-07-31,3164.68,-0.211996
7,AMZN,2020-08-31,3450.96,0.069751
8,AMZN,2020-09-30,3148.73,-0.081648
9,AMZN,2020-10-31,3036.15,-0.164316


In [ ]:
agg_df = monthly_h_values_df.groupby("Ticker").agg(startDate=("Date", "min"), endDate=("Date", "max")).reset_index()


In [ ]:
agg_df

,Ticker,startDate,endDate
0,AMZN,2020-01-31,2023-08-31


In [ ]:
agg_df.to_csv('sample_tickers_implied_H.csv', index=False)

In [ ]:
adjusted_close_value = stocks_data[stocks_data['Date'] == '2023-01-03']
adjusted_close_value

NameError: name 'stocks_data' is not defined

In [ ]:
# Get the descriptive statistics
description = h_values_df['TSLA'].describe()
print(description)

# Calculate specific percentiles
percentiles = h_values_df['TSLA'].quantile([0.25, 0.5, 0.75])
print(percentiles)

count       0
unique      0
top       NaN
freq      NaN
Name: TSLA, dtype: object
0.25   NaN
0.50   NaN
0.75   NaN
Name: TSLA, dtype: float64


In [ ]:
h_values_df

,TSLA,IBM
2023-01-03,NaN,0.087699
2023-01-04,NaN,0.039011
2023-01-05,NaN,0.311081
2023-01-06,NaN,0.063208
2023-01-09,NaN,0.052940
...,...,...
2023-12-22,NaN,NaN
2023-12-26,NaN,NaN
2023-12-27,NaN,NaN
2023-12-28,NaN,NaN
